In [1]:
from contrans import contrans
import numpy as np
import pandas as pd
import dotenv
import os
import json

In [2]:
ct = contrans()

In [5]:
bioguides =ct.get_bioguideIDs()
bio_df = pd.json_normalize(bioguides, record_path=['members'])
bio_df[['name', 'state', 'district', 'bioguideId']]

,name,state,district,bioguideId
0,"McIver, LaMonica",New Jersey,10.0,M001229
1,"Evans, Daniel J.",Washington,NaN,E000236
2,"Sasser, Jim",Tennessee,NaN,S000068
3,"Helmy, George S.",New Jersey,NaN,H001097
4,"Ruiz, Raul",California,25.0,R000599
5,"De La Cruz, Monica",Texas,15.0,D000594
6,"Menendez, Robert",New Jersey,NaN,M000639
7,"Carr, Bob",Michigan,8.0,C000178
8,"Nowak, Henry",New York,33.0,N000163
9,"Pascrell, Bill",New Jersey,9.0,P000096


In [8]:
ideo = ct.get_ideology()
members = ideo[['bioname', 'icpsr', 'party_code']]
members

,bioname,icpsr,party_code
0,"ROGERS, Mike Dennis",20301,200
1,"SEWELL, Terri",21102,100
2,"PALMER, Gary James",21500,200
3,"CARL, Jerry L.",22108,200
4,"MOORE, Barry",22140,200
...,...,...,...
443,"STEIL, Bryan",21970,200
444,"TIFFANY, Thomas P.",21989,200
445,"FITZGERALD, Scott",22115,200
446,"VAN ORDEN, Derrick",22370,200


In [9]:
votes = ct.get_votes()
votes

,congress,chamber,rollnumber,icpsr,cast_code,prob
0,118,House,1,14854,6,100.0
1,118,House,1,14863,6,98.2
2,118,House,1,14873,1,100.0
3,118,House,1,15029,1,100.0
4,118,House,1,15433,1,100.0
...,...,...,...,...,...,...
544855,118,Senate,582,42305,1,99.7
544856,118,Senate,582,49308,1,91.5
544857,118,Senate,582,49703,6,94.6
544858,118,Senate,582,90915,6,68.8


In [10]:
house = house[['rollnumber','icpsr', 'cast_code']]

In [24]:
house_mat = pd.merge(house, house, on='rollnumber')
house_mat

,rollnumber,icpsr_x,cast_code_x,icpsr_y,cast_code_y
0,1,14854,6,14854,6
1,1,14854,6,14863,6
2,1,14854,6,14873,1
3,1,14854,6,15029,1
4,1,14854,6,15433,1
...,...,...,...,...,...
211856531,1118,91980,1,31101,6
211856532,1118,91980,1,31102,9
211856533,1118,91980,1,39301,6
211856534,1118,91980,1,39307,6


In [25]:

house_mat['agree'] = house_mat['cast_code_x'] == house_mat['cast_code_y']
house_mat = house_mat.groupby(['icpsr_x', 'icpsr_y']).agg({'agree':'mean'})
house_mat = house_mat.reset_index()
house_mat

,icpsr_x,icpsr_y,agree
0,14854,14854,1.000000
1,14854,14863,0.830948
2,14854,14873,0.386404
3,14854,15029,0.412343
4,14854,15433,0.366726
...,...,...,...
200607,91980,31102,0.731664
200608,91980,31103,0.190805
200609,91980,39301,0.236136
200610,91980,39307,0.239714


In [17]:
agreement = pd.merge(house_mat, members,
                     left_on = 'icpsr_x',
                     right_on = 'icpsr')
agreement = agreement.rename({'bioname':'member1',
                              'party_code':'party1'}, axis=1)
agreement = agreement.drop(['icpsr_x'], axis=1)
agreement = pd.merge(agreement, members,
                     left_on = 'icpsr_y',
                     right_on = 'icpsr')
agreement = agreement.rename({'bioname':'member2',
                              'party_code':'party2'}, axis=1)
agreement = agreement[['member1', 'party1', 'member2', 'party2', 'agree']]
agreement

,member1,party1,member2,party2,agree
0,"ROGERS, Harold Dallas (Hal)",200,"ROGERS, Harold Dallas (Hal)",200,1.000000
1,"ROGERS, Harold Dallas (Hal)",200,"SMITH, Christopher Henry",200,0.830948
2,"ROGERS, Harold Dallas (Hal)",200,"HOYER, Steny Hamilton",100,0.386404
3,"ROGERS, Harold Dallas (Hal)",200,"KAPTUR, Marcia Carolyn (Marcy)",100,0.412343
4,"ROGERS, Harold Dallas (Hal)",200,"MFUME, Kweisi",100,0.366726
...,...,...,...,...,...
200607,"VAN DREW, Jefferson",200,"MASSIE, Thomas",200,0.731664
200608,"VAN DREW, Jefferson",200,"PAYNE, Donald, Jr.",100,0.190805
200609,"VAN DREW, Jefferson",200,"CLYBURN, James Enos",100,0.236136
200610,"VAN DREW, Jefferson",200,"SCOTT, Robert C.",100,0.239714


In [23]:
agreement[agreement['member1'].str.contains('GOOD,')].sort_values('agree',  ascending=False).head(30)

,member1,party1,member2,party2,agree
129269,"GOOD, Bob",200,"GOOD, Bob",200,1.000000
129324,"GOOD, Bob",200,"CRANE, Eli",200,0.944544
129147,"GOOD, Bob",200,"BIGGS, Andrew S.",200,0.941860
129292,"GOOD, Bob",200,"ROSENDALE, Matthew M., Sr.",200,0.937388
129187,"GOOD, Bob",200,"BURCHETT, Timothy",200,0.929338
129316,"GOOD, Bob",200,"BRECHEEN, Josh",200,0.918605
129178,"GOOD, Bob",200,"CLOUD, Michael",200,0.913238
129189,"GOOD, Bob",200,"CLINE, Benjamin",200,0.912343
129174,"GOOD, Bob",200,"NORMAN, Ralph",200,0.911449
129094,"GOOD, Bob",200,"PERRY, Scott",200,0.910555
